In [2]:
import pandas as pd
from sqlalchemy import create_engine, MetaData
from sqlalchemy import Column, Integer, String, Float, Date
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
import sqlalchemy

In [2]:
!rm hawaii.db

In [3]:
engine = create_engine('sqlite:///hawaii.db')

In [4]:
class measurement(Base):
    __tablename__ = 'measurement'
    id = Column(Integer, primary_key=True)
    station = Column(String(11))
    date = Column(String(10))
    prcp = Column(Float)
    tobs = Column(Integer) #temperature is in wholenumbers

class station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station = Column(String(11))
    name = Column(String(50))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [5]:
Base.metadata.create_all(engine)

In [6]:
# The ORM’s “handle” to the database is the Session.
from sqlalchemy.orm import Session
session = Session(engine)

In [7]:
df_meas = pd.read_csv("Resources/hawaii_measurements.csv")
df_station = pd.read_csv("Resources/hawaii_stations.csv")

In [10]:
display(df_meas.head())
display(df_station.head())
df_meas.loc[df_meas["station"]=='USC00519281' & df_meas["date"]>'2016-08-22']

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


TypeError: cannot compare a dtyped [object] array with a scalar of type [bool]

In [9]:
data_meas = df_meas.to_dict(orient='records')
data_station = df_station.to_dict(orient='records')

In [10]:
print(data_meas[:5])
print(data_station[:5])
df_meas.dtypes

[{'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'station': 'USC00519397', 'date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'station': 'USC00519397', 'date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'station': 'USC00519397', 'date': '2010-01-04', 'prcp': 0.0, 'tobs': 76}, {'station': 'USC00519397', 'date': '2010-01-06', 'prcp': nan, 'tobs': 73}]
[{'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elevation': 3.0}, {'station': 'USC00513117', 'name': 'KANEOHE 838.1, HI US', 'latitude': 21.4234, 'longitude': -157.8015, 'elevation': 14.6}, {'station': 'USC00514830', 'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'latitude': 21.5213, 'longitude': -157.8374, 'elevation': 7.0}, {'station': 'USC00517948', 'name': 'PEARL CITY, HI US', 'latitude': 21.3934, 'longitude': -157.9751, 'elevation': 11.9}, {'station': 'USC00518838', 'name': 'UPPER WAHIAWA 874.3, HI US', 'latitude': 21.4992, 'longitude': -158.0111, 'el

station     object
date        object
prcp       float64
tobs         int64
dtype: object

In [11]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)

In [12]:
table_meas = sqlalchemy.Table('measurement', metadata, autoload=True)
table_station = sqlalchemy.Table('station', metadata, autoload=True)

In [13]:
conn = engine.connect()
conn.execute(table_meas.insert(), data_meas)
conn.execute(table_station.insert(), data_station)